# FELIPE GRUMSER FRANCO VELHO
## MPV DA DISCIPLINA ENGENHARIA DE DADOS

O presente trabalho tem como objetivo criar um modelo de dados percorrendo pipelines de ETL utilizando a linguagem SQL. Ele está estruturado nas camadas bronze, prata e ouro. A carga de dados é feita na camada bronze, a análise da qualidade de manipulação dos dados ocorrerá na camada prata e as consultas na camada Gold.

A base de dados escolhida foi a publicamente disponibilizada pela IMDb (Internet Movie Database), que contém informações sobre filmes, séries e documentários. Com base nos dados disponibilizados, objetivou-se tratá-los a fim de obter, na camada Gold, mecanismos para responder perguntas sobre as relações de ano e gênero. As perguntas que se deseja responder são:
- Qual o ranking dos gêneros dos filmes lançados no ano X com base na média das suas notas?
- Quais as Top 10 melhores notas e seus respectivos ano e gênero?
- Dado um gênero X, qual o Top 50 melhores notas e seus respectivos anos?
- Quais são os anos com o maior número de votos, independente de gênero?


O trecho de código a seguir realiza a importação das bibliotecas necessárias para a manipulação e obtenção de dados. A biblioteca pandas, importada como pd, é utilizada para análise e manipulação de dados em formato tabular. A biblioteca io permite o manuseio de fluxos de dados em memória. Já a biblioteca requests possibilita a realização de requisições HTTP, sendo útil para acessar dados hospedados em fontes externas, como os arquivos disponibilizados pelo IMBd, via url. Para esse projeto, foi importada a tabela contendo o título original e primário das obras, seu tipo, se é conteúdo adulto, ano de início, tempo de exibição e gênero. A outra tabela importada contém a média de avaliações de cada título e o número de votos. Além disso, todos os filmes possuem um identificador "tconst". 

In [0]:
import pandas as pd
import io
import requests

In [0]:
url = "https://datasets.imdbws.com/title.basics.tsv.gz"
title_basics_df = pd.read_csv(url, compression='gzip', sep='\t')

title_basics_df.head()

<command-1370257541152233>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_basics_df = pd.read_csv(url, compression='gzip', sep='\t')


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short


In [0]:
url = "https://datasets.imdbws.com/title.ratings.tsv.gz"
title_ratings_df = pd.read_csv(url, compression='gzip', sep='\t')

title_ratings_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2146
1,tt0000002,5.5,291
2,tt0000003,6.5,2180
3,tt0000004,5.3,187
4,tt0000005,6.2,2924


Como mencionado anteriormente, o trabalho está dividido em categorias, começando pela camada bronze. Primeiramente, realizou-se a exclusão da camada para garantir que não houvesse nenhuma tabela previamente criada com o mesmo nome e a recriou.

In [0]:
%sql
DROP DATABASE bronze;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1370257541152238>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-1370257541152238>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("RFJPUCBEQVRBQkFTRSBicm9uemU=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/sp

In [0]:
%sql
CREATE DATABASE bronze;

A coluna isAdult apresentou um erro ao ser criada no dataframe Spark. Como os seus dados não seriam úteis para resoponder as perguntas, optou-se por removê-la para facilitar o processo.

In [0]:
title_basics_df = title_basics_df.drop(columns=["isAdult"])

Os blocos de código a seguir estão criando as tabelas "title_basics" e "title_ratings" na camada bronze do banco de dados. O Apache Spark permite manipulação de dados de forma eficiente, sendo essencial para a criação do dataframe. A primeira linha é responsável por, a partir do dataframe gerado pela importação do Pandas, criar um dataframe no Spark. Em seguida, o data frame do Spark é salvo na camada bronze como tabela, escrevendo por cima de possível arquivo de mesmo nome.

In [0]:
title_basics_spark_df = spark.createDataFrame(title_basics_df)
title_basics_spark_df.write.mode("overwrite").saveAsTable("bronze.title_basics")

In [0]:
title_ratings_spark_df = spark.createDataFrame(title_ratings_df)
title_ratings_spark_df.write.mode("overwrite").saveAsTable("bronze.title_ratings")

In [0]:
%sql
SELECT * FROM bronze.title_basics LIMIT 20

tconst,titleType,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,genres
tt0000001,short,Carmencita,Carmencita,1894,\N,1,"Documentary,Short"
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,1892,\N,5,"Animation,Short"
tt0000003,short,Poor Pierrot,Pauvre Pierrot,1892,\N,5,"Animation,Comedy,Romance"
tt0000004,short,Un bon bock,Un bon bock,1892,\N,12,"Animation,Short"
tt0000005,short,Blacksmith Scene,Blacksmith Scene,1893,\N,1,Short
tt0000006,short,Chinese Opium Den,Chinese Opium Den,1894,\N,1,Short
tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,1894,\N,1,"Short,Sport"
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,1894,\N,1,"Documentary,Short"
tt0000009,movie,Miss Jerry,Miss Jerry,1894,\N,45,Romance
tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,1895,\N,1,"Documentary,Short"


Aqui, visualizou-se os tipos de títulos disponíveis com o comando SELECT DISTINCT, que mostra os valores únicos da coluna selecionada. Como o presente trabalho vai analisar apenas filmes, apenas os tipos Movie, tvShort, short e tvMovie serão selecionados posteriormente.

In [0]:
%sql
SELECT DISTINCT titleType FROM bronze.title_basics;

titleType
tvSeries
tvMiniSeries
tvMovie
tvEpisode
movie
tvSpecial
video
videoGame
tvShort
short


In [0]:
%sql
SELECT * FROM bronze.title_ratings LIMIT 20

tconst,averageRating,numVotes
tt0000001,5.7,2146
tt0000002,5.5,291
tt0000003,6.5,2180
tt0000004,5.3,187
tt0000005,6.2,2924
tt0000006,5.0,210
tt0000007,5.3,906
tt0000008,5.4,2292
tt0000009,5.4,223
tt0000010,6.8,7922


Uma vez esgotado as ações a serem realizadas na camada bronze (importação de dados) cria-se de maneira similar a camada prata para a análise dos dados, verificando a sua consistência.

In [0]:
%sql
DROP DATABASE silver CASCADE

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1370257541152250>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-1370257541152250>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("RFJPUCBEQVRBQkFTRSBzaWx2ZXIgQ0FTQ0FERQ==").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /d

In [0]:
%sql
CREATE DATABASE silver

In [0]:
%sql
CREATE TABLE silver.title_ratings AS
SELECT * FROM bronze.title_ratings;

num_affected_rows,num_inserted_rows


O primeiro passo foi usar a explosão para separar os gêneros que originalmente estavam separados por vírgula na coluna "genre". Dessa forma, as obras que possuem mais de um gênero ocuparão mais de uma linha. Por esse motivo, o identificador único "tcosnt" perdeu a sua unicidade. Além disso, foram descatadas algumas colunas que não seriam relevantes para o projeto.

In [0]:
from pyspark.sql.functions import explode, split

title_genres_exploded_df = title_basics_spark_df.select(
    "tconst",
    "primaryTitle",
    "titleType",
    "startYear",
    explode(split("genres", ",")).alias("genre")
)

title_genres_exploded_df.show(10)


+---------+--------------------+---------+---------+-----------+
|   tconst|        primaryTitle|titleType|startYear|      genre|
+---------+--------------------+---------+---------+-----------+
|tt0000001|          Carmencita|    short|     1894|Documentary|
|tt0000001|          Carmencita|    short|     1894|      Short|
|tt0000002|Le clown et ses c...|    short|     1892|  Animation|
|tt0000002|Le clown et ses c...|    short|     1892|      Short|
|tt0000003|        Poor Pierrot|    short|     1892|  Animation|
|tt0000003|        Poor Pierrot|    short|     1892|     Comedy|
|tt0000003|        Poor Pierrot|    short|     1892|    Romance|
|tt0000004|         Un bon bock|    short|     1892|  Animation|
|tt0000004|         Un bon bock|    short|     1892|      Short|
|tt0000005|    Blacksmith Scene|    short|     1893|      Short|
+---------+--------------------+---------+---------+-----------+
only showing top 10 rows



In [0]:
title_genres_exploded_df.write.mode("overwrite").saveAsTable("silver.title_basics")

In [0]:
%sql
SELECT * FROM silver.title_basics LIMIT 100

tconst,primaryTitle,titleType,startYear,genre
tt0000001,Carmencita,short,1894,Documentary
tt0000001,Carmencita,short,1894,Short
tt0000002,Le clown et ses chiens,short,1892,Animation
tt0000002,Le clown et ses chiens,short,1892,Short
tt0000003,Poor Pierrot,short,1892,Animation
tt0000003,Poor Pierrot,short,1892,Comedy
tt0000003,Poor Pierrot,short,1892,Romance
tt0000004,Un bon bock,short,1892,Animation
tt0000004,Un bon bock,short,1892,Short
tt0000005,Blacksmith Scene,short,1893,Short


A partir desse ponto, as tabelas são submetidas a uma série de consultas, utilizando os comando SQL, para verificar a qualidade dos dados.

In [0]:
%sql
SELECT COUNT(*) 
FROM silver.title_basics 
WHERE tconst IS NULL;

count(1)
0


Não foram encontrados valores nulos em tconst.

In [0]:
%sql
SELECT *
FROM silver.title_basics
WHERE primaryTitle IS NULL OR TRIM(primaryTitle) = '' OR primaryTitle = '\N'


tconst,primaryTitle,titleType,startYear,genre
tt5337950,N,tvEpisode,2012,Comedy
tt5773048,null,tvEpisode,2015,Talk-Show
tt29387614,N,short,2011,Drama
tt29387614,N,short,2011,Music
tt29387614,N,short,2011,Short
tt33094763,null,tvEpisode,2019,Drama
tt31462159,null,tvEpisode,2024,Drama
tt4491564,N,short,1978,Short
tt19316378,N,tvEpisode,2010,Music
tt15700278,null,tvEpisode,2021,Talk-Show


Foram identificadas 44 linhas com inconsistências na coluna primaryTitle e optou-se por alterar esses valores para "Unknown".

In [0]:
%sql
UPDATE silver.title_basics
SET primaryTitle = 'Unknown'
WHERE (primaryTitle IS NULL OR primaryTitle = '\N' OR TRIM(primaryTitle) = '')

num_affected_rows
44


In [0]:
%sql
SELECT COUNT(*) 
FROM silver.title_basics
WHERE titleType IS NULL OR TRIM(titleType) = '' OR primaryTitle = '\\N';

count(1)
0


Não foram encontrados valores nulos em titleType.

In [0]:
%sql
SELECT COUNT(*)
FROM silver.title_basics 
WHERE startYear IS NULL OR startYear = '\\N';

count(1)
2097887


In [0]:
%sql
DELETE FROM silver.title_basics 
WHERE startYear IS NULL OR startYear = '\\N';

num_affected_rows
2097887


Optou-se por deletar todos os filmes com valores de startYear nulos já que o dado é essencial para a análise.

In [0]:
%sql
SELECT COUNT(*)
FROM silver.title_basics 
WHERE startYear = ' ';

count(1)
0


In [0]:
%sql
ALTER TABLE silver.title_basics 
ADD COLUMN startYear_int INT;

UPDATE silver.title_basics 
SET startYear_int = CAST(startYear AS INT);

num_affected_rows
16369220


Os valores de startYear estavam armazenados como string, sendo necessária a sua conversão para INT a fim de realizar as análises.

In [0]:
%sql
SELECT MIN(startYear_int), MAX(startYear_int) 
FROM silver.title_basics

min(startYear_int),max(startYear_int)
1874,2031


In [0]:
%sql
SELECT COUNT(*)
FROM silver.title_basics
WHERE startYear_int > 2024

count(1)
147621


Como o IMDB cataloga filmes que ainda não foram lançados, optou-se por excluir qualquer filme com startYear de 2025 em diante.

In [0]:
%sql
DELETE FROM silver.title_basics
WHERE startYear_int > 2024;

num_affected_rows
147621


In [0]:
%sql
SELECT COUNT(*) 
FROM silver.title_basics 
WHERE genre IS NULL OR genre = '\\N' OR TRIM(genre) = '';


count(1)
421303


In [0]:
%sql
DELETE FROM silver.title_basics 
WHERE genre IS NULL OR genre = '\\N' OR TRIM(genre) = '';

num_affected_rows
421303


Como gênero é um dado essencial para a análise, as tuplas sem esse dado foram deletadas.

In [0]:
%sql
SELECT genre, COUNT(*) 
FROM silver.title_basics 
GROUP BY genre 
ORDER BY COUNT(*) DESC;


genre,count(1)
Drama,2692487
Comedy,1965639
Talk-Show,1228376
Short,1185111
Documentary,1041027
News,980943
Romance,937987
Family,715930
Reality-TV,575510
Animation,496381


Há um número expressivo de filmes em cada um dos gênero

In [0]:
%sql
SHOW TABLES IN silver;

database,tableName,isTemporary
silver,title_basics,false
silver,title_ratings,false


Terminada as verificações na tabela com os títulos, iniciou-se as na tabela com as avaliações

In [0]:
%sql
SELECT *
FROM silver.title_ratings LIMIT 50;

tconst,averageRating,numVotes
tt0000001,5.7,2146
tt0000002,5.5,291
tt0000003,6.5,2180
tt0000004,5.3,187
tt0000005,6.2,2924
tt0000006,5.0,210
tt0000007,5.3,906
tt0000008,5.4,2292
tt0000009,5.4,223
tt0000010,6.8,7922


In [0]:
%sql
SELECT COUNT(*)
FROM silver.title_ratings
WHERE tconst IS NULL OR TRIM(tconst) = '' OR tconst = '\\N';

count(1)
0


Não há valores nulos em tconst

In [0]:
%sql
SELECT COUNT(*)
FROM silver.title_ratings
WHERE averageRating IS NULL OR TRIM(averageRating) = '';

count(1)
0


Não há valores nulos em avarageRating

In [0]:
%sql
SELECT COUNT(*)
FROM silver.title_ratings
WHERE numVotes IS NULL OR TRIM(numVotes) = '';

count(1)
0


Não há valores nulos em numVotes

In [0]:
%sql
SELECT COUNT(*)
FROM silver.title_ratings
WHERE averageRating < 0 OR averageRating > 10;

count(1)
0


Não há notas fora do intervalo de 0 a 10

In [0]:
%sql
SELECT COUNT(*)
FROM silver.title_ratings
WHERE numVotes = 0;

count(1)
0


Não há filmes com 0 número de votos

In [0]:
%sql
SELECT COUNT(*)
FROM silver.title_ratings r
LEFT JOIN silver.title_basics b ON r.tconst = b.tconst
WHERE b.tconst IS NULL;

count(1)
33294


Verificando a correspondência dos títulos nas duas tabelas com o comando JOIN.

As duas tabelas serão mergidas usando o comando JOIN pelo valor tconst

In [0]:
%sql
CREATE TABLE silver.title_joined AS
SELECT 
    b.tconst,
    b.primaryTitle,
    b.titleType,
    b.startYear,
    b.genre,
    r.averageRating,
    r.numVotes
FROM 
    silver.title_basics b
JOIN 
    silver.title_ratings r
ON 
    b.tconst = r.tconst;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT *
FROM silver.title_joined LIMIT 100;

tconst,primaryTitle,titleType,startYear,genre,averageRating,numVotes
tt0000008,Edison Kinetoscopic Record of a Sneeze,short,1894,Documentary,5.4,2292
tt0000008,Edison Kinetoscopic Record of a Sneeze,short,1894,Short,5.4,2292
tt0000015,Around a Cabin,short,1894,Animation,6.1,1271
tt0000015,Around a Cabin,short,1894,Comedy,6.1,1271
tt0000015,Around a Cabin,short,1894,Short,6.1,1271
tt0000017,Italienischer Bauerntanz,short,1895,Documentary,4.6,379
tt0000017,Italienischer Bauerntanz,short,1895,Short,4.6,379
tt0000019,The Clown Barber,short,1898,Comedy,5.2,35
tt0000019,The Clown Barber,short,1898,Short,5.2,35
tt0000051,The Bohemian Encampment,short,1896,Documentary,3.5,39


Considerando que as atividades a serem realizadas na camada prata foram encerradas, a camada ouro será criada a seguir.

In [0]:
%sql
DROP DATABASE gold CASCADE

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1370257541152297>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-1370257541152297>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("RFJPUCBEQVRBQkFTRSBnb2xkIENBU0NBREU=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /datab

In [0]:
%sql
CREATE DATABASE gold

O código a seguir cria a tabela final, que será usada para a consulta. Ele filtra os títulos para incluir apenas os tipos movie, tvShort, short e tvMovie, que serão os considerados na análise. O startYear teve seu nome alterado para year e foi calculada a média de avaliações e a soma do número de votos por ano e gênero. Para isso, o código agrupou startYear e genre, juntando os registros em que esse valores são iguais. Por fim, ordena o resultado por ano e gênero.

In [0]:
%sql
CREATE TABLE gold.genre_analysis AS
SELECT
    startYear AS year,
    genre,
    AVG(CAST(averageRating AS DOUBLE)) AS avgRating, 
    SUM(CAST(numVotes AS INT)) AS totalVotes
FROM
    silver.title_joined
WHERE
    titleType IN ('movie', 'tvShort', 'short', 'tvMovie')
GROUP BY
    startYear,
    genre
ORDER BY
    startYear,
    genre;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT *
FROM gold.genre_analysis
WHERE year > 1980 LIMIT 500;

year,genre,avgRating,totalVotes
1981,Action,5.807232704402517,1943670
1981,Adult,5.490948275862069,17224
1981,Adventure,6.039784946236559,2135619
1981,Animation,6.554587155963303,200103
1981,Biography,6.846052631578947,162441
1981,Comedy,5.845045045045046,956122
1981,Crime,5.961089494163424,474711
1981,Documentary,7.03962962962963,37504
1981,Drama,6.147728860936409,1603074
1981,Family,6.4010989010989015,44161


In [0]:
%sql
UPDATE gold.genre_analysis
SET avgRating = ROUND(avgRating, 3);

num_affected_rows
3073


Os códigos a seguir são feitas as consultas a fim de responder as perguntas originalmente elaboradas

In [0]:
%sql
SELECT DISTINCT *
FROM gold.genre_analysis
WHERE year = 1980
ORDER BY avgRating DESC;

year,genre,avgRating,totalVotes
1980,Documentary,6.997,29097
1980,Biography,6.764,771122
1980,History,6.734,101770
1980,Music,6.651,87941
1980,Short,6.473,26113
1980,Animation,6.456,47625
1980,Family,6.399,114855
1980,War,6.29,137122
1980,Fantasy,6.193,1560092
1980,Romance,6.118,332014


Aqui foi feito um recorte de um ano específico utilizando o comando _WHERE year = 1980_, para verificar as melhores avaliações por gênero. O _ORDER BY avgRating DESC_ ordena da maior para a menor nota. Observa-se notas mais elevadas pertencem ao gênero documentário. Horror é o último colocado.

In [0]:
%sql
SELECT DISTINCT *
FROM gold.genre_analysis
WHERE year = 1990
ORDER BY avgRating DESC;

year,genre,avgRating,totalVotes
1990,Reality-TV,7.467,67
1990,Documentary,7.116,47469
1990,Music,7.048,83877
1990,History,6.703,75740
1990,Biography,6.694,1598286
1990,Animation,6.597,139980
1990,Musical,6.565,77287
1990,Short,6.434,24207
1990,Family,6.347,944198
1990,War,6.338,109235


10 anos depois, no ano de 1990, documentário caiu uma posição e foi substituído por Reality TV, categoria que nem existia em 1990. No entanto, o número de votos é muito baixo. O fato do filtro ter selecionado apenas filmes na criação da tabela, indica um possível erro na categorização do Reality TV.
Observa-se que o valor da média de documentário aumentou, enquanto a média de horro decaiu, mantendo o gênero em último colocado.

In [0]:
%sql
SELECT *
FROM gold.genre_analysis
ORDER BY avgRating DESC
LIMIT 10;

year,genre,avgRating,totalVotes
2015,Game-Show,9.9,10
1903,Sci-Fi,9.0,11
2003,News,8.65,66
1983,Game-Show,8.6,16
2022,News,8.6,2924
2005,Talk-Show,8.5,9
1960,Reality-TV,8.5,21
2005,Reality-TV,8.45,156
1993,News,8.4,31
1998,Talk-Show,8.4,142


Aqui são apresentadas as maiores notas. No entanto, é possível perceber um padrão de que há poucou votos em cada tupla, sendo uma métrica menos precisa pela pequena amostragem.

In [0]:
%sql
SELECT *
FROM gold.genre_analysis
WHERE totalVotes > 50000
ORDER BY avgRating DESC
LIMIT 10;

year,genre,avgRating,totalVotes
2008,News,7.92,56033
2024,Short,7.695,64345
2023,Short,7.45,233516
1996,Music,7.442,219790
1961,Sport,7.413,90552
1995,Music,7.374,123715
2022,Short,7.374,208517
1989,Music,7.362,242762
2024,Documentary,7.354,509314
2022,Documentary,7.305,849212


Para obter uma amostragem maior, filtrou-se os resultados em que o número de votos foi superior à 50000

In [0]:
%sql
SELECT *
FROM gold.genre_analysis
WHERE genre = 'Drama'
ORDER BY avgRating DESC
LIMIT 50;

year,genre,avgRating,totalVotes
2024,Drama,6.893,8043960
1888,Drama,6.8,6
2016,Drama,6.699,19822044
2023,Drama,6.696,12459619
2022,Drama,6.685,15410269
2012,Drama,6.674,19320243
2014,Drama,6.66,25873631
2017,Drama,6.658,19581096
2011,Drama,6.634,20291455
2015,Drama,6.617,19560153


Foi feita uma análise do gênero drama, avaliando quais os anos com as melhoras notas no gênero. Mais uma vez, o ano de 1888, com um número pequeno de notas, aparece com nota elevada, possivelmente por uma pequena amostragem.

In [0]:
%sql
SELECT *
FROM gold.genre_analysis
WHERE genre = 'Action'
ORDER BY avgRating DESC
LIMIT 50;

year,genre,avgRating,totalVotes
2012,Action,6.361,15826469
1956,Action,6.335,48174
2024,Action,6.279,6812101
2014,Action,6.266,20360863
2016,Action,6.24,16698187
2011,Action,6.23,16529661
1943,Action,6.217,49185
2013,Action,6.215,16320595
1958,Action,6.202,77693
2015,Action,6.2,14161583


Quanto ao gênero ação, percebe-se uma maior avaliação nos filmes mais recentes.

In [0]:
%sql
SELECT year, SUM(totalVotes) AS total_votes_per_year
FROM gold.genre_analysis
GROUP BY year
ORDER BY total_votes_per_year DESC
LIMIT 50;

year,total_votes_per_year
2014,130251052
2013,125331991
2016,109451559
2011,109298254
2012,108119573
2010,107872792
2008,105299012
2017,103790684
2015,103690345
2009,102858945


Por fim, o código acima mostra os anos com maior número de votos. Com o advento da internet e da popularização dos aparelhos eletrônicos, é esperado que filmes mais recentes apresentem maior número de votos devido ao momento que foram lançados. Não obstantante, o consumo de filmes é atemporal, especialmente na era do streaming, em que há acesso de forma fácil a diversos títulos. Assim, obras do século passado são consumidas nos dias atuais.

# AUTOAVALIAÇÃO
De maneira geral, avalio positivamente a minha experiência elaborando o trabalho. Incialmente, busquei base de dados referentes a premiações do mundo da música, como o Grammy. Pensei em fazer análises com gênero, mas nesse caso do álbum. No entanto, buscando no Kaggle tive dificuldades de encontrar uma base de dados consistida e tive um pouco de dificuldade com o CSV. Portando, resolvi ir para a do IMDb, que estava mais consolidada e que me permitiria fazer um trabalho parecido com o que originalmente pensei.

O primeiro problema que encontrei foi o com a coluna "isAdult". Pensei em corrigí-lo em vez de apagá-lo, mas como o dado não seria de meu interesse, optei por excluí-lo. Não sei se foi a ação mais prudente, considerando que futuramente poderia fazer o uso dele.
Outra questão que me ocorreu foi o fato da chave primária tconst ter perdido a sua unicidade quando explodi os gêneros. Julguei passível de ocorrer considerando que naõ afetaria os objetivos do projeto.

Tive certas dificuldades para elaborar alguns códigos, mas consultando consegui elaborá-los.

Ao final, fui percebendo possíveis falhas no modelo, especialmente considerando a amostragem dos dados. Filmes mais antigos apresentam poucas notas. Talvez fosse prudente limitar o período de análise ou excluir filmes com poucas notas. 

A criação do MPV foi muito interressante para colocar em prática o conteúdo aprendido e obter informações de maneira categórica de dados brutos, após o seu tratamente, foi muito enriquecedor. Consegui ter um gostinho do poder da análise de dados.